In [1]:
import os


In [2]:
%pwd

'e:\\desktop\\learning\\ML\\projects\\end to end kidney stone classification\\End-to-End-kidney-stone-classification\\End-to-End-kidney-stone-classification-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\desktop\\learning\\ML\\projects\\end to end kidney stone classification\\End-to-End-kidney-stone-classification\\End-to-End-kidney-stone-classification-'

In [5]:
#pip install pillow --user

In [6]:
from dataclasses import dataclass
from pathlib import Path

#entity
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from src.CNNclassifier.constants import *
from src.CNNclassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-01-02 21:44:54,591: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-02 21:44:54,596: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-02 21:44:54,598: INFO: common: created directory at: artifacts]
[2024-01-02 21:44:54,601: INFO: common: created directory at: artifacts\training]
Found 600 images belonging to 2 classes.
Found 2400 images belonging to 2 classes.
150/150 [==============================] - 680s 5s/step - loss: 11.5471 - accuracy: 0.5754 - val_loss: 22.3622 - val_accuracy: 0.5068


In [ ]:
#conda uninstall --force pillow


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



PackagesNotFoundError: The following packages are missing from the target environment:
  - pillow




In [ ]:
#pip install pillow

Note: you may need to restart the kernel to use updated packages.


Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are missing from the target environment:
  - pillow




In [ ]:
#conda install Pillow


Note: you may need to restart the kernel to use updated packages.Collecting package metadata (current_repodata.json): ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\mohammed\anaconda3\envs\kidney

  added / updated specs:
    - pillow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pillow-10.0.1              |   py38h045eedc_0         781 KB
    xz-5.4.5                   |       h8cc25b3_0         593 KB
    ------------------------------------------------------------
                                           Total:         1.3 MB

The following NEW packages will be INSTALLED:

  freetype           pkgs/main/win-64::freetype-2.12.1-ha860e81_0
  giflib             pkgs/main/win-64::giflib-5.2.1-h8cc25b3_3
  jpeg               pkgs/main/win-64::jpeg-9e-h2bbff1b_1
  lerc               pkgs/main/win-64::lerc-3.0-hd77b12b_



==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda


